<a href="https://colab.research.google.com/github/tamaskecskemeti/financial_nlp/blob/main/nlp_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install required packages written in requirements
!pip install -r requirements.txt

In [2]:
from transformers import GPT2LMHeadModel, set_seed, AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import pipeline
import torch
import torch.nn.functional as F
import itertools
import numpy as np
from pathlib import Path

In [3]:
set_seed(42)

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "cpu"
device

'cuda:0'

# Function creator

In [5]:
def generate_text_from_input(tokenizer, model, input_text):
  input_ids = tokenizer.encode(input_text, return_tensors="pt")

  out = model.generate(input_ids,
                     max_new_tokens=100,
                     num_beams=5,
                     no_repeat_ngram_size=4,
                     top_k=50,
                     do_sample=True,
                     top_p=0.9,
                     temperature=1,
                     early_stopping=True,
                     pad_token_id=tokenizer.eos_token_id)

  out_text = list(map(tokenizer.decode, out))[0]

  return out_text

In [6]:
# rouge scores for a reference/generated sentence pair
# source google seq2seq source code.

# supporting function
def _split_into_words(sentences):
  """Splits multiple sentences into words and flattens the result"""
  return list(itertools.chain(*[_.split(" ") for _ in sentences]))

# supporting function
def _get_word_ngrams(n, sentences):
  """Calculates word n-grams for multiple sentences.
  """
  assert len(sentences) > 0
  assert n > 0

  words = _split_into_words(sentences)
  return _get_ngrams(n, words)

# supporting function
def _get_ngrams(n, text):
  """Calcualtes n-grams.
  Args:
    n: which n-grams to calculate
    text: An array of tokens
  Returns:
    A set of n-grams
  """
  ngram_set = set()
  text_length = len(text)
  max_index_ngram_start = text_length - n
  for i in range(max_index_ngram_start + 1):
    ngram_set.add(tuple(text[i:i + n]))
  return ngram_set

def rouge_n(reference_sentences, evaluated_sentences, n=2):
  """
  Args:
    evaluated_sentences: The sentences that have been picked by the summarizer
    reference_sentences: The sentences from the referene set
    n: Size of ngram.  Defaults to 2.
  Returns:
    recall rouge score(float)
  Raises:
    ValueError: raises exception if a param has len <= 0
  """
  if len(evaluated_sentences) <= 0 or len(reference_sentences) <= 0:
    raise ValueError("Collections must contain at least 1 sentence.")

  evaluated_ngrams = _get_word_ngrams(n, evaluated_sentences)
  reference_ngrams = _get_word_ngrams(n, reference_sentences)
  reference_count = len(reference_ngrams)
  evaluated_count = len(evaluated_ngrams)

  # gets the overlapping ngrams between evaluated and reference
  overlapping_ngrams = evaluated_ngrams.intersection(reference_ngrams)
  overlapping_count = len(overlapping_ngrams)

  # handle edge case. This isn't mathematically correct, but it's good enough
  if evaluated_count == 0:
    precision = 0.0
  else:
    precision = overlapping_count / evaluated_count

  if reference_count == 0:
    recall = 0.0
  else:
    recall = overlapping_count / reference_count

  f1_score = 2.0 * ((precision * recall) / (precision + recall + 1e-8))

  # just returning recall count in rouge, useful for our purpose
  return recall

# English model

In [10]:
# some text to test the model
# text_en = np.loadtxt("train_data_en.txt")
text_en = "The Turing test, originally called the imitation game by Alan Turing in 1950 is a test of a machine's ability to exhibit intelligent behaviour equivalent to, or indistinguishable from, that of a human."

In [9]:
tokenizer_1_en = AutoTokenizer.from_pretrained("ai-forever/mGPT")
model_1_en = GPT2LMHeadModel.from_pretrained("ai-forever/mGPT")
tokenizer_2_en = AutoTokenizer.from_pretrained("gpt2")
model_2_en = GPT2LMHeadModel.from_pretrained("gpt2")

In [11]:
generated_text_1_en = generate_text_from_input(tokenizer_1_en, model_1_en, text_en)
generated_text_1_en

'The Turing test, originally called the imitation game by Alan Turing in 1950 is a test of a machine\'s ability to exhibit intelligent behaviour equivalent to, or indistinguishable from, that of a human. It is a form of the Turing test in which a machine is able to demonstrate that it is capable of reasoning in a way similar to that of humans.\nThe Turing test was first developed by Alan Turing, a British mathematician and computer scientist, in 1950, to test his ability to imitate a human. The test was designed to test Turing\'s ability to reason in a manner similar to human reasoning. The Turing test was originally called the "imitation game" by Turing'

In [12]:
generated_text_2_en = generate_text_from_input(tokenizer_2_en, model_2_en, text_en)
generated_text_2_en

"The Turing test, originally called the imitation game by Alan Turing in 1950 is a test of a machine's ability to exhibit intelligent behaviour equivalent to, or indistinguishable from, that of a human.\n\nIn the Turing test, the Turing test is a computer's ability to discriminate between two or more possible outcomes. The Turing test is not a test of human intelligence, but a test of the Turing test.\n\nThe Turing test\n\nIt is important to note that the Turing test does not determine whether or not a machine is intelligent or not. Rather, it determines whether or not the Turing test can be used to determine whether a machine is smarter or not.\n\nTuring is"

In [13]:
# the reference text is used to evaluate the generated text
ref_text_en = "The Turing test, originally called the imitation game by Alan Turing in 1950 is a test of a machine's ability to exhibit intelligent behaviour equivalent to, or indistinguishable from, that of a human. Turing proposed that a human evaluator would judge natural language conversations between a human and a machine designed to generate human-like responses. The evaluator would be aware that one of the two partners in conversation was a machine, and all participants would be separated from one another. The conversation would be limited to a text-only channel, such as a computer keyboard and screen, so the result would not depend on the machine's ability to render words as speech. If the evaluator could not reliably tell the machine from the human, the machine would be said to have passed the test. The test results would not depend on the machine's ability to give correct answers to questions, only on how closely its answers resembled those a human would give. Since the Turing test is a test of indistinguishability in performance capacity, the verbal version generalizes naturally to all of human performance capacity, verbal as well as nonverbal (robotic)."

In [14]:
# the rouge value can be between 0 and 1. The higher value is better
print(rouge_n(ref_text_en, generated_text_1_en))

0.6260162601626016


In [15]:
# the rouge value can be between 0 and 1. The higher value is better
print(rouge_n(ref_text_en, generated_text_2_en))

0.6056910569105691


# Hungarian model

In [16]:
# some text in hungarian to test the model
text_hu = "A Turing-teszt, amelyet eredetileg Alan Turing 1950-ben imitációs játéknak nevezett el, egy olyan teszt, amely azt vizsgálja, hogy egy gép képes-e az emberi viselkedéssel egyenértékű vagy attól megkülönböztethetetlen intelligens viselkedést tanúsítani."


In [17]:
generated_text_1_hu = generate_text_from_input(tokenizer_1_en, model_1_en, text_hu)

In [18]:
generated_text_1_hu

'A Turing-teszt, amelyet eredetileg Alan Turing 1950-ben imitációs játéknak nevezett el, egy olyan teszt, amely azt vizsgálja, hogy egy gép képes-e az emberi viselkedéssel egyenértékű vagy attól megkülönböztethetetlen intelligens viselkedést tanúsítani. A tesztet úgy tervezték, hogy a Turing-teszttel ellentétben ne tegyék lehetővé, hogy az emberi agy ne tudjon azonosulni a gépekkel.\nA tesztet az 1950-es évek elején végezték el, és a tesztet olyan számítógépeken végezték, amelyek nem rendelkeztek a Turing-testekkel. Ezek a számítógépek a következők voltak:\nA tesztrendszer.'

In [19]:
tokenizer_2_hu = AutoTokenizer.from_pretrained("NYTK/PULI-GPT-2")
model_2_hu = GPT2LMHeadModel.from_pretrained("NYTK/PULI-GPT-2")

generated_text_2_hu = generate_text_from_input(tokenizer_2_hu, model_2_hu, text_hu)

In [20]:
generated_text_2_hu

'A Turing-teszt, amelyet eredetileg Alan Turing 1950-ben imitációs játéknak nevezett el, egy olyan teszt, amely azt vizsgálja, hogy egy gép képes-e az emberi viselkedéssel egyenértékű vagy attól megkülönböztethetetlen intelligens viselkedést tanúsítani. A Turing-tesztet az 1950-es években kezdték el kidolgozni, és azóta is széles körben alkalmazzák.\nA Turing teszt lényege, hogy a gép képes legyen az emberi viselkedéshez hasonló vagy attól eltérő intelligenciát tanúsítani. Ez azt jelenti, hogy a teszt során a gépnek képesnek kell lennie az emberi viselkedésre hasonló vagy attól különböző viselkedést tanúsítani, függetlenül attól, hogy az adott gép képes volt-e arra, hogy az emberi viselkedéstől eltérő viselkedést tanúsítson'

In [21]:
# the reference text is used to evaluate the generated text
ref_text_hu = "A Turing-teszt, amelyet eredetileg Alan Turing 1950-ben imitációs játéknak nevezett el, egy olyan teszt, amely azt vizsgálja, hogy egy gép képes-e az emberi viselkedéssel egyenértékű vagy attól megkülönböztethetetlen intelligens viselkedést tanúsítani. Turing azt javasolta, hogy egy emberi értékelő értékelje az ember és egy emberhez hasonló válaszok generálására tervezett gép közötti természetes nyelvű beszélgetéseket. Az értékelő tisztában lenne azzal, hogy a két beszélgetőpartner közül az egyik egy gép, és a résztvevők el lennének választva egymástól. A beszélgetés kizárólag szöveges csatornára korlátozódna, például számítógépes billentyűzetre és képernyőre, így az eredmény nem függne attól, hogy a gép képes-e a szavakat beszédként megjeleníteni. Ha az értékelő nem tudná megbízhatóan megkülönböztetni a gépet az embertől, akkor a gép átmenne a teszten. A teszt eredménye nem függne attól, hogy a gép képes-e helyes válaszokat adni a kérdésekre, csak attól, hogy a válaszai mennyire hasonlítanak az emberi válaszokhoz. Mivel a Turing-teszt a teljesítőképesség megkülönböztethetetlenségének tesztje, a verbális változat természetesen általánosítható az emberi teljesítőképesség egészére, a verbális és a nem verbális (robotikus) teljesítőképességre egyaránt."

In [22]:
# the rouge value can be between 0 and 1. The higher value is better
print(rouge_n(ref_text_hu, generated_text_1_hu))

0.5828025477707006


In [23]:
# the rouge value can be between 0 and 1. The higher value is better
print(rouge_n(ref_text_hu, generated_text_2_hu))

0.6273885350318471


# Fine-tune models

In [24]:
def load_dataset(file_path, tokenizer, block_size = 128):
  dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
  return dataset


def load_data_collator(tokenizer):
  data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )
  return data_collator


def train(input_path,
          model_name,
          output_path,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):

  tokenizer = AutoTokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(input_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_path)
  model = GPT2LMHeadModel.from_pretrained(model_name)
  model.save_pretrained(output_path)

  training_args = TrainingArguments(
          output_dir=output_path,
          overwrite_output_dir=False,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [ ]:
input_path_en = "train_data_en.txt"
output_path_en = "result_en"
model_name_en = "gpt2"

input_path_hu = "train_data_hu.txt"
output_path_hu = "result_hu"
model_name_hu = "NYTK/PULI-GPT-2"
per_device_train_batch_size = 6
num_train_epochs = 256
save_steps = 250

In [ ]:
train(
    input_path=input_path_en,
    model_name=model_name_en,
    output_path=output_path_en,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

In [ ]:
train(
    input_path=input_path_hu,
    model_name=model_name_hu,
    output_dir=output_path_hu,
    overwrite_output_dir=False,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

In [ ]:
tokenizer_trained_en = AutoTokenizer.from_pretrained(output_path_en)
model_trained_en = GPT2LMHeadModel.from_pretrained(output_path_en)

generated_text_en = generate_text_from_input(tokenizer_trained_en, model_trained_en, text_en)

In [ ]:
generated_text_en

'The economics is:\na\nprice\ntheoretically\nconceivable\nwould be\na\nthing\n.\nFrom\nsure\n,\nthat\non\nApple\nvery\nquickly\nout\nmust\nfind\nsomething\na\n29\n'

In [ ]:
tokenizer_trained_hu = AutoTokenizer.from_pretrained(output_path_hu)
model_trained_hu = GPT2LMHeadModel.from_pretrained(output_path_hu)

generated_text_hu = generate_text_from_input(tokenizer_trained_hu, model_trained_hu, text_hu)

In [ ]:
# the rouge value can be between 0 and 1. The higher value is better
print(rouge_n(ref_text_en, generated_text_en))

In [ ]:
# the rouge value can be between 0 and 1. The higher value is better
print(rouge_n(ref_text_hu, generated_text_hu))